In [1]:
import datetime
import os

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

from skimage import color, exposure

from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

sns.set()
%load_ext tensorboard

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
HOME = '/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix_road_signs'
%cd $HOME

/content/drive/My Drive/Colab Notebooks/matrix/dw_matrix_road_signs


In [0]:
train_db = pd.read_pickle('data/train.p')
test_db = pd.read_pickle('data/test.p')

X_train, y_train = train_db['features'], train_db['labels']
X_test, y_test = test_db['features'], test_db['labels']

In [4]:
sign_names = pd.read_csv('data/dw_signnames.csv')
sign_names.head()

,a,b
0,0,Speed limit (20km/h)
1,1,Speed limit (30km/h)
2,2,Speed limit (50km/h)
3,3,Speed limit (60km/h)
4,4,Speed limit (70km/h)


In [0]:
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
cat_num = y_train.shape[1]

In [7]:
def get_cnn_v1(input_shape, cat_num, verbose=False):
    model = Sequential([Conv2D(filters=64, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                        Flatten(),
                        Dense(cat_num, activation='softmax')])
    if verbose:
        model.summary()

    return model


cnn_v1 = get_cnn_v1(input_shape, cat_num, True)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 30, 30, 64)        1792      
_________________________________________________________________
flatten (Flatten)            (None, 57600)             0         
_________________________________________________________________
dense (Dense)                (None, 43)                2476843   
Total params: 2,478,635
Trainable params: 2,478,635
Non-trainable params: 0
_________________________________________________________________


In [8]:
def train_model(model, X_train, y_train, params_fit=dict()):

    logdir = os.path.join('logs', datetime.datetime.now().strftime('%Y%m%d_%H%M%S'))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])
    model.fit(X_train, 
              y_train,
              batch_size=params_fit.get('batch_size', 128),
              epochs=params_fit.get('epochs', 5),
              verbose=params_fit.get('verbose', 1),
              validation_data=params_fit.get('validation_data', (X_train, y_train)),
              callbacks=[tensorboard_callback])

    return model

model_trained = train_model(cnn_v1, X_train, y_train)

Epoch 1/5
272/272 [==============================] - 7s 25ms/step - loss: 21.1410 - accuracy: 0.7725 - val_loss: 0.2157 - val_accuracy: 0.9500
Epoch 2/5
272/272 [==============================] - 6s 23ms/step - loss: 0.1960 - accuracy: 0.9538 - val_loss: 0.1232 - val_accuracy: 0.9707
Epoch 3/5
272/272 [==============================] - 6s 23ms/step - loss: 0.1243 - accuracy: 0.9693 - val_loss: 0.0857 - val_accuracy: 0.9778
Epoch 4/5
272/272 [==============================] - 6s 23ms/step - loss: 0.0941 - accuracy: 0.9764 - val_loss: 0.0711 - val_accuracy: 0.9825
Epoch 5/5
272/272 [==============================] - 6s 23ms/step - loss: 0.0943 - accuracy: 0.9792 - val_loss: 0.1108 - val_accuracy: 0.9747


In [0]:
def predict(model, X_test, y_test, scoring=accuracy_score):
    y_test_norm = np.argmax(y_test, axis=1)
    y_pred_prob = model.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn(input_shape, cat_num):
    model = Sequential([Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                        MaxPool2D(),
                        Dropout(0.3),

                        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                        MaxPool2D(),
                        Dropout(0.3),

                        Flatten(),

                        Dense(1024, activation='relu'),
                        Dropout(0.3),

                        Dense(1024, activation='relu'),
                        Dropout(0.3),

                        Dense(cat_num, activation='softmax')])
    return model

In [11]:
def train_and_predict(model, X_train, y_train, X_test, y_test):
    model_trained = train_model(model, X_train, y_train)
    return predict(model_trained, X_test, y_test)

# train_and_predict(get_cnn(input_shape, cat_num), X_train, y_train, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 12s 45ms/step - loss: 2.3776 - accuracy: 0.4112 - val_loss: 0.4694 - val_accuracy: 0.8598
Epoch 2/5
272/272 [==============================] - 12s 43ms/step - loss: 0.5820 - accuracy: 0.8224 - val_loss: 0.1191 - val_accuracy: 0.9720
Epoch 3/5
272/272 [==============================] - 12s 43ms/step - loss: 0.3022 - accuracy: 0.9095 - val_loss: 0.0622 - val_accuracy: 0.9845
Epoch 4/5
272/272 [==============================] - 12s 43ms/step - loss: 0.1948 - accuracy: 0.9410 - val_loss: 0.0264 - val_accuracy: 0.9929
Epoch 5/5
272/272 [==============================] - 12s 43ms/step - loss: 0.1507 - accuracy: 0.9559 - val_loss: 0.0218 - val_accuracy: 0.9943


0.962358276643991

In [0]:
def get_model(input_shape, cat_num, params):
    model = Sequential([Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
                        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
                        MaxPool2D(),
                        Dropout(params['dropout_cnn_0']),

                        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
                        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
                        MaxPool2D(),
                        Dropout(params['dropout_cnn_1']),

                        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
                        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
                        MaxPool2D(),
                        Dropout(params['dropout_cnn_2']),

                        Flatten(),

                        Dense(1024, activation='relu'),
                        Dropout(params['dropout_dense_0']),

                        Dense(1024, activation='relu'),
                        Dropout(params['dropout_dense_1']),

                        Dense(cat_num, activation='softmax')])
    return model

In [0]:
def func_obj(params):
    model = get_model(input_shape, cat_num, params)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    model.fit(X_train, 
              y_train,
              batch_size=int(params.get('batch_size', 128)),
              epochs=params.get('epochs', 5),
              verbose=params.get('verbose', 0)
    )

    score = model.evaluate(X_test, y_test, verbose=0)
    accuracy = score[1]
    print(f'params={params}')
    print(f'accuracy={accuracy}')

    return {'loss': -accuracy, 'status': STATUS_OK, 'model': model}

In [0]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_0': hp.uniform('dropout_cnn_0', 0.3, 0.5),
    'dropout_cnn_1': hp.uniform('dropout_cnn_1', 0.3, 0.5),
    'dropout_cnn_2': hp.uniform('dropout_cnn_2', 0.3, 0.5),
    'dropout_dense_0': hp.uniform('dropout_dense_0', 0.3, 0.7),
    'dropout_dense_1': hp.uniform('dropout_dense_1', 0.3, 0.7),
}

In [25]:
best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30, 
    Trials()
)

params={'batch_size': 100.0, 'dropout_cnn_0': 0.4879239947277033, 'dropout_cnn_1': 0.41844465850576257, 'dropout_cnn_2': 0.30965372112329087, 'dropout_dense_0': 0.4021819483430512, 'dropout_dense_1': 0.5484462572265736}
accuracy=0.9546485543251038
params={'batch_size': 110.0, 'dropout_cnn_0': 0.4454706127504861, 'dropout_cnn_1': 0.4624384690395012, 'dropout_cnn_2': 0.33913598936283884, 'dropout_dense_0': 0.5740434204430944, 'dropout_dense_1': 0.6296515937419154}
accuracy=0.8337868452072144
params={'batch_size': 160.0, 'dropout_cnn_0': 0.36067887838440155, 'dropout_cnn_1': 0.46008918518034936, 'dropout_cnn_2': 0.43839249380954826, 'dropout_dense_0': 0.3246246348241844, 'dropout_dense_1': 0.3160098003597073}
accuracy=0.9435374140739441
params={'batch_size': 120.0, 'dropout_cnn_0': 0.32139805632785357, 'dropout_cnn_1': 0.3682460950330647, 'dropout_cnn_2': 0.35017506728862746, 'dropout_dense_0': 0.636572271397232, 'dropout_dense_1': 0.571017519489049}
accuracy=0.9678004384040833
params={'b